In [ ]:
!pip install unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 2.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.3/299.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.2/376.2 kB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.5/166.5 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.1/117.1 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.2/821.2 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.7/155.7 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 91.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23

In [11]:
import json


data=json.load(open('/content/fixed_missing_fields_dataset_2000.json','r'))

In [12]:
data

[{'input': "Can you extract details about the Asus ROG Strix? It's categorized under electronics, costs $1106, and is made by Amazon. (Item 0)",
  'output': {'name': 'Asus ROG Strix',
   'price': '$1106',
   'category': 'electronics',
   'manufacturer': 'Amazon'}},
 {'input': "Can you extract details about the iPad Air? It's categorized under audio, costs $1344, and is made by Dell. (Item 1)",
  'output': {'name': 'iPad Air',
   'price': '$1344',
   'category': 'audio',
   'manufacturer': 'Dell'}},
 {'input': 'Describe the Google Pixel 7 for me — it costs $712, is under smartphones, and made by Dell. (Item 2)',
  'output': {'name': 'Google Pixel 7',
   'price': '$712',
   'category': 'smartphones',
   'manufacturer': 'Dell'}},
 {'input': 'Need specs for a Bose QuietComfort 45 — comes under audio, tagged at $2429, and manufactured by Dell.',
  'output': {'name': 'Bose QuietComfort 45',
   'price': '$2429',
   'category': 'audio',
   'manufacturer': 'Dell'}},
 {'input': 'Product alert: G

In [13]:
from unsloth import FastLanguageModel

model_name='unsloth/Llama-3.1-8B-Instruct'

In [14]:
import torch

print(torch.cuda.is_available())

True


In [15]:
model,tokenizer=FastLanguageModel.from_pretrained(
    model_name,
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
    load_in_8bit = False,
)

==((====))==  Unsloth 2025.7.8: Fast Llama patching. Transformers: 4.53.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

In [16]:
model = FastLanguageModel.get_peft_model(
    model,
    r=64,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha=128,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)

Unsloth 2025.7.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [17]:
tokenizer.chat_template

'{{- bos_token }}\n{%- if custom_tools is defined %}\n    {%- set tools = custom_tools %}\n{%- endif %}\n{%- if not tools_in_user_message is defined %}\n    {%- set tools_in_user_message = true %}\n{%- endif %}\n{%- if not date_string is defined %}\n    {%- set date_string = "26 Jul 2024" %}\n{%- endif %}\n{%- if not tools is defined %}\n    {%- set tools = none %}\n{%- endif %}\n\n{#- This block extracts the system message, so we can slot it into the right place. #}\n{%- if messages[0][\'role\'] == \'system\' %}\n    {%- set system_message = messages[0][\'content\']|trim %}\n    {%- set messages = messages[1:] %}\n{%- else %}\n    {%- set system_message = "" %}\n{%- endif %}\n\n{#- System message + builtin tools #}\n{{- "<|start_header_id|>system<|end_header_id|>\\n\\n" }}\n{%- if builtin_tools is defined or tools is not none %}\n    {{- "Environment: ipython\\n" }}\n{%- endif %}\n{%- if builtin_tools is defined %}\n    {{- "Tools: " + builtin_tools | reject(\'equalto\', \'code_interp

In [18]:
EOS_TOKEN=tokenizer.eos_token

In [19]:
for i in data:
  print(i)
  break

{'input': "Can you extract details about the Asus ROG Strix? It's categorized under electronics, costs $1106, and is made by Amazon. (Item 0)", 'output': {'name': 'Asus ROG Strix', 'price': '$1106', 'category': 'electronics', 'manufacturer': 'Amazon'}}


In [20]:
tokenizer.chat_template = """
{% for message in messages %}
<|start_header_id|>{{ message['role'] }}<|end_header_id|>
{{ message['content'] }}
{% endfor %}
<|eot_id|>
"""


In [21]:
def format_prompt(sample):
  input=sample['input']
  output=sample['output']
  text=[{
    'role':'system','content':'You are a helpful and intelligent AI assistant.'
},
      {
      'role':'user','content':input,
  },
      {
      'role':'assistant','content':output
  }]
  formatted = tokenizer.apply_chat_template(
        text,
        add_generation_prompt=False,
        tokenize=False
    )

  return formatted

In [22]:
for i in data:
  print(i)
  break

{'input': "Can you extract details about the Asus ROG Strix? It's categorized under electronics, costs $1106, and is made by Amazon. (Item 0)", 'output': {'name': 'Asus ROG Strix', 'price': '$1106', 'category': 'electronics', 'manufacturer': 'Amazon'}}


In [23]:
formatted_data=[format_prompt(i) for i in data ]

In [24]:
formatted_data={'text':formatted_data}

In [25]:
from datasets import Dataset


In [26]:
formatted_data = Dataset.from_dict(formatted_data)


In [27]:
formatted_data

Dataset({
    features: ['text'],
    num_rows: 2000
})

In [28]:
formatted_data['text'][0]

"\n<|start_header_id|>system<|end_header_id|>\nYou are a helpful and intelligent AI assistant.\n<|start_header_id|>user<|end_header_id|>\nCan you extract details about the Asus ROG Strix? It's categorized under electronics, costs $1106, and is made by Amazon. (Item 0)\n<|start_header_id|>assistant<|end_header_id|>\n{'name': 'Asus ROG Strix', 'price': '$1106', 'category': 'electronics', 'manufacturer': 'Amazon'}\n<|eot_id|>"

In [29]:
from trl import SFTConfig, SFTTrainer
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = formatted_data,
    dataset_text_field = "text",
    max_seq_length = 2048,
    packing = False,
    args = SFTConfig(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/2000 [00:00<?, ? examples/s]

In [30]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 2,000 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 167,772,160 of 8,198,033,408 (2.05% trained)


Step,Training Loss
1,4.903300
2,4.868500
3,4.905100
4,5.034600
5,4.850400
6,3.758100
7,3.254900
8,2.505600
9,2.030200
10,1.747300


In [31]:
FastLanguageModel.for_inference(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
        (layers): ModuleList(
          (0): LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear

In [43]:
messages = [
    {"role": "system", "content": "You are an AI that extracts product information in JSON format. Only respond based on the latest user message. Do not hallucinate missing details."},
    {"role": "user", "content": "Buy our Samsung A15 for 25 from nearest Samsung Electronics Store"},
]

# Step 2: Tokenize using chat template
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt"
).to("cuda")

# Step 3: Generate output deterministically
outputs = model.generate(
    input_ids=inputs,
    max_new_tokens=128,
    do_sample=False,
    temperature=0.0,
    top_p=1.0,
)


In [44]:
tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)

["\nsystem\nYou are an AI that extracts product information in JSON format. Only respond based on the latest user message. Do not hallucinate missing details.\nuser\nBuy our Samsung A15 for 25 from nearest Samsung Electronics Store\nassistant\n{'name': 'Samsung A15', 'price': '$25', 'category': '','manufacturer': 'Samsung Electronics'}\n"]

In [45]:
model.save_pretrained_gguf("Finetuned.gguf", tokenizer, quantization_method = "q4_k_m")

Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which might take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 6.0G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 2.86 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...



 41%|████      | 13/32 [00:01<00:01, 12.67it/s]
We will save to Disk and not RAM now.

100%|██████████| 32/32 [06:33<00:00, 12.29s/it]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving Finetuned.gguf/pytorch_model-00001-of-00004.bin...
Unsloth: Saving Finetuned.gguf/pytorch_model-00002-of-00004.bin...
Unsloth: Saving Finetuned.gguf/pytorch_model-00003-of-00004.bin...
Unsloth: Saving Finetuned.gguf/pytorch_model-00004-of-00004.bin...
Done.


Unsloth: Converting llama model. Can use fast conversion = False.


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q4_k_m'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: CMAKE detected. Finalizing some steps for installation.
Unsloth: [1] Converting model at Finetuned.gguf into f16 GGUF format.
The output location will be /content/Finetuned.gguf/unsloth.F16.gguf
This might take 3 minutes...
INFO:hf-to-gguf:Loading model: Finetuned.gguf
INFO:hf-to-gguf:Model architecture: LlamaForCausalLM
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:rope_freqs.weight,           torch.float32 --> F32, shape = {64}
INFO:hf-to-gguf:gguf: loading model weight map from 'pytorch_model.bin.index.